In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
from beamtest_analysis_helper import etroc2_analysis_helper
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted
import hist
import mplhep as hep
hep.style.use('CMS')

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data

chip_names = ["ET2_W36_IP7_13_HV210V_offset24","ET2_EPIR_1_1_HV210V_offset24", "ET2_CNM_1_3_HV210V_offset6"]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 WB W36 IP7-13 HV210V OS:24","(Trigger) ETROC2 BB EPIR 1-1 HV210V OS:24", "ETROC2 BB CNM 1-3 HV210V OS:6"]

chip_labels= ["1","0","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# path_pattern = f"*2023-09-21_Array_Test_Results/SelfTrigger_bottom_Readout_topbottom_1"
# path_pattern = f"./testbeam_sep24/SelfTrigger_ET2_CNM_BATCH_1_3_Readout_ET2_EPIR_BATCH1_1_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_loop_*.pqt"
path_pattern = "./highpower_offset6/SelfTrigger_ET2_EPIR_BATCH1_1_Readout_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_offset6_highpower_FINALRUN_loop_*.pqt"

helper = etroc2_analysis_helper()

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()

## Event selections

In [ ]:
files = glob(path_pattern)
files = natsorted(files)

dataframes = []

for ifile in files:
    tmp_df = pd.read_parquet(ifile)
    if tmp_df.empty:
        continue

    # Group the DataFrame by 'evt' and count unique 'board' values in each group
    unique_board_counts = tmp_df.groupby('evt')['board'].nunique()

    ## event has three unique board ID
    event_numbers_with_three_unique_boards = unique_board_counts[unique_board_counts == 3].index
    subset_df = tmp_df[tmp_df['evt'].isin(event_numbers_with_three_unique_boards)]
    subset_df.reset_index(inplace=True, drop=True)

    del tmp_df
    if subset_df.empty: 
        continue
    
    dataframes.append(subset_df)

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

In [ ]:
df.info()

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
del h_inclusive

In [ ]:
helper.making_heatmap_byPandas(df, chip_labels, chip_figtitles, "inclusive")

In [ ]:
helper.making_3d_heatmap_byPandas(df, chip_labels, chip_figtitles, "inclusive")

### Event counter and 2D heatmap based on WB pixel selection

In [ ]:
pix_info = [1, 15, 6]
simple_filtered_df = helper.find_maximum_event_combination(df, pix_info)

In [ ]:
helper.making_heatmap_byPandas(simple_filtered_df, chip_labels, chip_figtitles, figtitle_tag="WB pixel (15, 6)")

In [ ]:
del simple_filtered_df

### Pixel ID selection based on the event counter

In [ ]:
pix_dict = {
    # board ID: [row, col]
    0: [ 2, 6],
    1: [15, 6],
    2: [ 0, 0],
    3: [ 3, 5],
}

filtered_group = helper.pixel_filter(df, pix_dict)
filtered_group = helper.singlehit_event_clear_func(filtered_group)
filtered_group.info()

In [ ]:
del df

In [ ]:
h_pix_selected = helper.return_hist(filtered_group, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (15, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (2, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (3, 5)")

In [ ]:
del h_pix_selected

### TDC cut

In [ ]:
# Define custom filtering criteria for each board
tdc_cuts = {    
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    0: [199, 205,   0, 1100,   0, 600],
    1: [200, 210, 275,  350,   0, 600],
    2: [160, 220,   0, 1100,   0, 600],
    3: [189, 195,   0, 1000,   0, 600],
}

tdc_filtered_df = helper.tdc_event_selection(filtered_group, tdc_cuts)
tdc_filtered_df = helper.singlehit_event_clear_func(tdc_filtered_df)
tdc_filtered_df.info()

In [ ]:
del filtered_group

In [ ]:
def sort_filter(group):
    return group.sort_values(by=['board'], ascending=True)

def distance_filter(group, distance):
    board0_row = group[(group["board"] == 0)]
    board3_row = group[(group["board"] == 3)]
    board0_col = group[(group["board"] == 0)]
    board3_col = group[(group["board"] == 3)]

    if not board0_row.empty and not board3_row.empty and not board0_col.empty and not board3_col.empty:
        row_index_diff = abs(board0_row["row"].values[0] - board3_row["row"].values[0])
        col_index_diff = abs(board0_col["col"].values[0] - board3_col["col"].values[0])
        return row_index_diff < distance and col_index_diff < distance
    else:
        return False
    
# tmp_group = selected_subset_df.groupby('evt')
# filtered_simple_group = tmp_group.filter(simple_filter, board=1, row=15, col=6)
# filtered_simple_group.reset_index(inplace=True, drop=True)
# del tmp_group

# grouped = filtered_simple_group.groupby('evt')
# sorted_filtered_simple_group = grouped.apply(sort_filter)
# sorted_filtered_simple_group.reset_index(inplace=True, drop=True)
# sorted_filtered_simple_group
# del grouped

# grouped = sorted_filtered_simple_group.groupby('evt')
# dis_simple_group = grouped.filter(distance_filter, distance=2)
# dis_simple_group

# test_group = dis_simple_group.groupby(['board', 'row', 'col'])
# test = test_group.size().reset_index(name='count')
# test.to_csv('test.csv', index=False)

# del filtered_simple_group,sorted_filtered_simple_group,grouped,dis_simple_group, test_group

In [ ]:
h_tdc_selection = helper.return_hist(tdc_filtered_df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="after_tdc_cut", title_tag=", Pixel (15, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="", title_tag=", Pixel (2, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="", title_tag=", Pixel (2, 5) after TDC cut")

In [ ]:
del h_tdc_selection

## Convert TDC code to TDC time in [ns]

In [ ]:
selected_df = tdc_filtered_df

pix_rows = []
pix_cols = []
fit_params = []
cal_means = {boardID:{} for boardID in chip_labels}

for boardID in chip_labels:
    groups = selected_df[selected_df['board'] == int(boardID)].groupby(['row', 'col'])
    for (row, col), group in groups:
        
        cal_mean = group['cal'].mean()
        cal_means[boardID][(row, col)] = cal_mean

cal_means

In [ ]:
bin0 = (3.125/cal_means["0"][(2, 6)])
bin1 = (3.125/cal_means["1"][(15, 6)])
bin3 = (3.125/cal_means["3"][(3, 5)])

toa_in_time_b0 = 12.5 - selected_df[selected_df['board'] == 0]['toa'] * bin0
toa_in_time_b1 = 12.5 - selected_df[selected_df['board'] == 1]['toa'] * bin1
toa_in_time_b3 = 12.5 - selected_df[selected_df['board'] == 3]['toa'] * bin3

tot_in_time_b0 = (2*selected_df[selected_df['board'] == 0]['tot'] - np.floor(selected_df[selected_df['board'] == 0]['tot']/32)) * bin0
tot_in_time_b1 = (2*selected_df[selected_df['board'] == 1]['tot'] - np.floor(selected_df[selected_df['board'] == 1]['tot']/32)) * bin1
tot_in_time_b3 = (2*selected_df[selected_df['board'] == 3]['tot'] - np.floor(selected_df[selected_df['board'] == 3]['tot']/32)) * bin3

d = {
    'evt': selected_df['evt'].unique(),
    'toa_b0': toa_in_time_b0.to_numpy(),
    'tot_b0': tot_in_time_b0.to_numpy(),
    'toa_b1': toa_in_time_b1.to_numpy(),
    'tot_b1': tot_in_time_b1.to_numpy(),
    'toa_b3': toa_in_time_b3.to_numpy(),
    'tot_b3': tot_in_time_b3.to_numpy(),
}

df_in_time = pd.DataFrame(data=d)
del d, selected_df, tdc_filtered_df

df_in_time.info()

## Traditional Time Walk Correction

In [ ]:
del_toa_b0 = (0.5*(df_in_time['toa_b1'] + df_in_time['toa_b3']) - df_in_time['toa_b0']).values
del_toa_b1 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b3']) - df_in_time['toa_b1']).values
del_toa_b3 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b1']) - df_in_time['toa_b3']).values

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b3'].values, del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)    
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, poly_func_b3(df_in_time['tot_b3'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
coeffs, corr_toas = helper.iterative_timewalk_correction(df_in_time, 3, 3)

In [ ]:
poly_func_b0 = np.poly1d(coeffs[0])
poly_func_b1 = np.poly1d(coeffs[1])
poly_func_b3 = np.poly1d(coeffs[2])

nth_del_toa_b0 = (0.5*(corr_toas[1] + corr_toas[2]) - corr_toas[0])
nth_del_toa_b1 = (0.5*(corr_toas[0] + corr_toas[2]) - corr_toas[1])
nth_del_toa_b3 = (0.5*(corr_toas[0] + corr_toas[1]) - corr_toas[2])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  nth_del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)    
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  nth_del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  nth_del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, poly_func_b3(df_in_time['tot_b3'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
corrTOA_b0 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b1 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b3 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

corrTOA_b0.fill(corr_toas[0])
corrTOA_b1.fill(corr_toas[1])
corrTOA_b3.fill(corr_toas[2])

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b0, ax=axes[0], lw=2)
axes[0].set_xlabel('Time Walk Corrected TOA [ns]')
axes[0].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b1, ax=axes[1], lw=2)
axes[1].set_xlabel('Time Walk Corrected TOA [ns]')
axes[1].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b3, ax=axes[2], lw=2)
axes[2].set_xlabel('Time Walk Corrected TOA [ns]')
axes[2].set_ylabel('Entries')

plt.tight_layout()

In [ ]:
tmp_dict = {
    'evt': df_in_time['evt'].values,
    'corr_toa_b0': corr_toas[0],
    'corr_toa_b1': corr_toas[1],
    'corr_toa_b3': corr_toas[2],
}

df_in_time_corr = pd.DataFrame(tmp_dict)
del tmp_dict
df_in_time_corr

In [ ]:
diff_b01 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b1']
diff_b03 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b3']
diff_b13 = df_in_time_corr['corr_toa_b1'] - df_in_time_corr['corr_toa_b3']

dTOA_b01 = hist.Hist(hist.axis.Regular(64, diff_b01.mean().round(2)-0.8, diff_b01.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b03 = hist.Hist(hist.axis.Regular(64, diff_b03.mean().round(2)-0.8, diff_b03.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b13 = hist.Hist(hist.axis.Regular(64, diff_b13.mean().round(2)-0.8, diff_b13.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

## These cuts only for a good fitting
diff_b01 = diff_b01[(diff_b01 > diff_b01.mean().round(2)-0.8) & (diff_b01 < diff_b01.mean().round(2)+0.8)]
diff_b03 = diff_b03[(diff_b03 > diff_b03.mean().round(2)-0.8) & (diff_b03 < diff_b03.mean().round(2)+0.8)]
diff_b13 = diff_b13[(diff_b13 > diff_b13.mean().round(2)-0.8) & (diff_b13 < diff_b13.mean().round(2)+0.8)]

dTOA_b01.fill(diff_b01)
dTOA_b03.fill(diff_b03)
dTOA_b13.fill(diff_b13)

### Fit using exponnorm in scipy.stats

In [ ]:
from scipy.stats import exponnorm

fit_params_exponnorm = []

range1 = (diff_b01.mean().round(2)-0.8, diff_b01.mean().round(2)+0.8)
range2 = (diff_b03.mean().round(2)-0.8, diff_b03.mean().round(2)+0.8)
range3 = (diff_b13.mean().round(2)-0.8, diff_b13.mean().round(2)+0.8)

fig, axes = plt.subplots(1, 3, figsize=(30, 10))

hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 - Board 1', loc="right", size=25)
hep.histplot(dTOA_b01, ax=axes[0], density=True, lw=2)
params = exponnorm.fit(diff_b01)
fit_params_exponnorm.append(params)
x_range = np.linspace(range1[0], range1[1], 500)
axes[0].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[0].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 0 - Board 3', loc="right", size=25)
hep.histplot(dTOA_b03, ax=axes[1], density=True, lw=2)
params = exponnorm.fit(diff_b03)
fit_params_exponnorm.append(params)
x_range = np.linspace(range2[0], range2[1], 500)
axes[1].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[1].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 1 - Board 3', loc="right", size=25)
hep.histplot(dTOA_b13, ax=axes[2], density=True, lw=2)
params = exponnorm.fit(diff_b13)
fit_params_exponnorm.append(params)
x_range = np.linspace(range3[0], range3[1], 500)
axes[2].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[2].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend()

plt.tight_layout()

In [ ]:
res_b3,err_b3 = helper.return_resolution_ps(fit_params_exponnorm[1][2], 0,
                                            fit_params_exponnorm[2][2], 0,
                                            fit_params_exponnorm[0][2], 0)
res_b1,err_b1 = helper.return_resolution_ps(fit_params_exponnorm[0][2], 0,
                                            fit_params_exponnorm[2][2], 0,
                                            fit_params_exponnorm[1][2], 0)
res_b0,err_b0 = helper.return_resolution_ps(fit_params_exponnorm[0][2], 0,
                                            fit_params_exponnorm[1][2], 0,
                                            fit_params_exponnorm[2][2], 0)

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

### Fit using lmfit ExponentialGaussianModel

In [ ]:
from lmfit.models import ExponentialGaussianModel

fit_params_lmfit = []

# Create a CrystalBallModel object
mod = ExponentialGaussianModel()
fig, axes = plt.subplots(1, 3, figsize=(30, 10))


edges = 0.5*(dTOA_b01.axes[0].edges[1:] + dTOA_b01.axes[0].edges[:-1])
pars = mod.guess(dTOA_b01.values(), x=edges)
out = mod.fit(dTOA_b01.values(), pars, x=edges)
fit_params_lmfit.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 - Board 1', loc="right", size=25)
hep.histplot(dTOA_b01, ax=axes[0], density=False, lw=2)
axes[0].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[0].legend(fontsize=20, loc='upper right')


edges = 0.5*(dTOA_b03.axes[0].edges[1:] + dTOA_b03.axes[0].edges[:-1])
pars = mod.guess(dTOA_b03.values(), x=edges)
out = mod.fit(dTOA_b03.values(), pars, x=edges)
fit_params_lmfit.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 0 - Board 3', loc="right", size=25)
hep.histplot(dTOA_b03, ax=axes[1], density=False, lw=2)
axes[1].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[1].legend(fontsize=20, loc='upper right')


edges = 0.5*(dTOA_b13.axes[0].edges[1:] + dTOA_b13.axes[0].edges[:-1])
pars = mod.guess(dTOA_b13.values(), x=edges)
out = mod.fit(dTOA_b13.values(), pars, x=edges)
fit_params_lmfit.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 1 - Board 3', loc="right", size=25)
hep.histplot(dTOA_b13, ax=axes[2], density=False, lw=2)
axes[2].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[2].legend(fontsize=20, loc='upper right')

plt.tight_layout()

In [ ]:
res_b0,err_b0 = helper.return_resolution_ps(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1])
res_b1,err_b1 = helper.return_resolution_ps(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1])
res_b3,err_b3 = helper.return_resolution_ps(fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[0][0], fit_params_lmfit[0][1])

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

## Optimized Time Walk Correction -  Pairwise Neural Network

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input

In [ ]:
def return_dense_model(numpars=2, print_summary=False):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(8, activation='relu', name='dense1',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(input)
    output = Dense(1, activation='linear', name='output',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    model  = Model(inputs=[input], outputs=output, name="simple_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    if(print_summary): print(model.summary())
    return model

model = return_dense_model(print_summary=True)

del model

In [ ]:
ensemble_count = 10
data_tag = 'tb_sep28_offset24_triggeroffset6'
extra_tag = "CALNarrow_TOA275to350forWB_NoTOTcut"

In [ ]:
def etroc_regression_using_NN(
        input_df: pd.DataFrame,
        variables: list,
        data_tag: str,
        extra_tag: str,
        board_tag: str,
        ensemble_count: int,
        figure_title: str,
        do_plotting: bool,
    ):
    filename = f'{data_tag}_weights_{extra_tag}_{board_tag}'

    for en_idx in range(ensemble_count):
        model = return_dense_model(numpars=2)
        checkpointer = ModelCheckpoint(f'models/NNRun{en_idx}_{filename}.hdf5', verbose=0, save_best_only=True, monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
        
        history = model.fit(
            input_df[[variables[0], variables[1]]].values, 
            (input_df[variables[2]]-input_df[variables[2]]).values, 
            validation_split=0.4, 
            epochs=150,
            callbacks=[checkpointer,term,escb],
            verbose=0)
        
        del model

        if (do_plotting):
            #plot the loss and validation loss of the dataset
            fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
            hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
            ax.set_title(f'Model {en_idx}, {figure_title}', loc="right", size=25)
            plt.plot(history.history['loss'], label='mse')
            plt.plot(history.history['val_loss'], label='val_mse')
            plt.yscale("log")
            plt.legend()
            plt.savefig(f"models/NNRun{en_idx}_{filename}.png")

### Run NN Training (skip if you don't want to do training)

In [ ]:
vars = ['tot_b0', 'tot_b1', 'toa_b0', 'toa_b1']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b01", ensemble_count, figure_title='Board 0 - Board 1', do_plotting=True)

vars = ['tot_b0', 'tot_b3', 'toa_b0', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b03", ensemble_count, figure_title='Board 0 - Board 3', do_plotting=True)

vars = ['tot_b1', 'tot_b3', 'toa_b1', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b13", ensemble_count, figure_title='Board 1 - Board 3', do_plotting=True)

### Call trained models

In [ ]:
model_b01 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b01'
for en_idx in range(ensemble_count):
    model_b01.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
    else: Y_pred += model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
del model_b01
Y_pred_b01 = Y_pred/ensemble_count

model_b03 = return_dense_model(numpars=2)
for en_idx in range(ensemble_count):
    model_b03.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
del model_b03
Y_pred_b03 = Y_pred/ensemble_count

model_b13 = return_dense_model(numpars=2)
for en_idx in range(ensemble_count):
    model_b13.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b13.predict(df_in_time[['tot_b1',  'tot_b3']].values, verbose=0).flatten()
del model_b13
Y_pred_b13 = Y_pred/ensemble_count

In [ ]:
data_b01 = (df_in_time['toa_b0']-df_in_time['toa_b1']).values-Y_pred_b01
data_b03 = (df_in_time['toa_b0']-df_in_time['toa_b3']).values-Y_pred_b03
data_b13 = (df_in_time['toa_b1']-df_in_time['toa_b3']).values-Y_pred_b13

## These cuts only for a good fitting
data_b01 = data_b01[(data_b01 > data_b01.mean().round(2)-0.5) & (data_b01 < data_b01.mean().round(2)+0.5)]
data_b03 = data_b03[(data_b03 > data_b03.mean().round(2)-0.5) & (data_b03 < data_b03.mean().round(2)+0.5)]
data_b13 = data_b13[(data_b13 > data_b13.mean().round(2)-0.5) & (data_b13 < data_b13.mean().round(2)+0.5)]

dTOA_NN_b01 = hist.Hist(hist.axis.Regular(50, data_b01.mean().round(2)-0.5, data_b01.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b03 = hist.Hist(hist.axis.Regular(50, data_b03.mean().round(2)-0.5, data_b03.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b13 = hist.Hist(hist.axis.Regular(50, data_b13.mean().round(2)-0.5, data_b13.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_NN_b01.fill(data_b01)
dTOA_NN_b03.fill(data_b03)
dTOA_NN_b13.fill(data_b13)

### scipy exponnorm fit with NN outputs

In [ ]:
from scipy.stats import exponnorm, norm

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
fit_params_exponnorm_NN = []

range1 = (data_b01.mean().round(2)-0.5, data_b01.mean().round(2)+0.5)
range2 = (data_b03.mean().round(2)-0.5, data_b03.mean().round(2)+0.5)
range3 = (data_b13.mean().round(2)-0.5, data_b13.mean().round(2)+0.5)
## ----------------------------------------------------


hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 - Board 1', loc="right", size=25)
hep.histplot(dTOA_NN_b01, ax=axes[0], density=True, lw=2)
params = exponnorm.fit(data_b01)
fit_params_exponnorm_NN.append(params)
x_range = np.linspace(range1[0], range1[1], 500)
axes[0].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[0].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend(fontsize=20, loc='upper right')
## ----------------------------------------------------


hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 0 - Board 3', loc="right", size=25)
hep.histplot(dTOA_NN_b03, ax=axes[1], density=True, lw=2)
params = exponnorm.fit(data_b03)
fit_params_exponnorm_NN.append(params)
x_range = np.linspace(range2[0], range2[1], 500)
axes[1].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[1].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend(fontsize=20, loc='upper right')
## ----------------------------------------------------

  
hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 - Board 3', loc="right", size=25)
hep.histplot(dTOA_NN_b13, ax=axes[2], density=True, lw=2)
params = exponnorm.fit(data_b13)
fit_params_exponnorm_NN.append(params)
x_range = np.linspace(range3[0], range3[1], 500)
axes[2].plot(x_range, exponnorm.pdf(x_range, *params), 'r--', lw=2, label=fr'$\mu:{params[1]:.3f}, \sigma: {abs(params[2]):.3f}$')
axes[2].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend(fontsize=20, loc='upper right')

In [ ]:
res_b3,err_b3 = helper.return_resolution_ps(fit_params_exponnorm_NN[1][2], 0,
                                            fit_params_exponnorm_NN[2][2], 0,
                                            fit_params_exponnorm_NN[0][2], 0)
res_b1,err_b1 = helper.return_resolution_ps(fit_params_exponnorm_NN[0][2], 0,
                                            fit_params_exponnorm_NN[2][2], 0,
                                            fit_params_exponnorm_NN[1][2], 0)
res_b0,err_b0 = helper.return_resolution_ps(fit_params_exponnorm_NN[0][2], 0,
                                            fit_params_exponnorm_NN[1][2], 0,
                                            fit_params_exponnorm_NN[2][2], 0)

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

### Fit using lmfit ExponentialGaussianModel with NN outputs

In [ ]:
from lmfit.models import ExponentialGaussianModel

fit_params_lmfit_NN = []

# Create a CrystalBallModel object
mod = ExponentialGaussianModel()
fig, axes = plt.subplots(1, 3, figsize=(30, 10))


edges = 0.5*(dTOA_NN_b01.axes[0].edges[1:] + dTOA_NN_b01.axes[0].edges[:-1])
pars = mod.guess(dTOA_NN_b01.values(), x=edges)
out = mod.fit(dTOA_NN_b01.values(), pars, x=edges)
fit_params_lmfit_NN.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 - Board 1', loc="right", size=25)
hep.histplot(dTOA_NN_b01, ax=axes[0], density=False, lw=2)
axes[0].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[0].legend(fontsize=20, loc='upper right')


edges = 0.5*(dTOA_NN_b03.axes[0].edges[1:] + dTOA_NN_b03.axes[0].edges[:-1])
pars = mod.guess(dTOA_NN_b03.values(), x=edges)
out = mod.fit(dTOA_NN_b03.values(), pars, x=edges)
fit_params_lmfit_NN.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 0 - Board 3', loc="right", size=25)
hep.histplot(dTOA_NN_b03, ax=axes[1], density=False, lw=2)
axes[1].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[1].legend(fontsize=20, loc='upper right')


edges = 0.5*(dTOA_NN_b13.axes[0].edges[1:] + dTOA_NN_b13.axes[0].edges[:-1])
pars = mod.guess(dTOA_NN_b13.values(), x=edges)
out = mod.fit(dTOA_NN_b13.values(), pars, x=edges)
fit_params_lmfit_NN.append([out.params['sigma'].value, out.params['sigma'].stderr])
hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 1 - Board 3', loc="right", size=25)
hep.histplot(dTOA_NN_b13, ax=axes[2], density=False, lw=2)
axes[2].plot(edges, out.best_fit, 'r--', lw=2, label=fr"$\mu:{out.params['center'].value:.3f}, \sigma: {abs(out.params['sigma'].value):.3f}$")
axes[2].legend(fontsize=20, loc='upper right')

plt.tight_layout()

In [ ]:
res_b0,err_b0 = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1])
res_b1,err_b1 = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1])
res_b3,err_b3 = helper.return_resolution_ps(fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1])

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

### OTWC Performance Plots

In [ ]:
model_b01 = return_dense_model(numpars=2)
model_b03 = return_dense_model(numpars=2)
model_b13 = return_dense_model(numpars=2)
model_b01.load_weights(f'models/sep27_weights_{0}_b01.hdf5')
nn_del_b01 = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
model_b03.load_weights(f'models/sep27_weights_{0}_b03.hdf5')
nn_del_b03 = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
model_b13.load_weights(f'models/sep27_weights_{0}_b13.hdf5')
nn_del_b13 = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
for en_idx in range(1, ensemble_count):
    model_b01.load_weights(f'models/sep27_weights_{en_idx}_b01.hdf5')
    nn_del_b01 += model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
    model_b03.load_weights(f'models/sep27_weights_{en_idx}_b03.hdf5')
    nn_del_b03 += model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
    model_b13.load_weights(f'models/sep27_weights_{en_idx}_b13.hdf5')
    nn_del_b13 += model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
nn_del_b01 = nn_del_b01/ensemble_count
nn_del_b03 = nn_del_b03/ensemble_count
nn_del_b13 = nn_del_b13/ensemble_count
del model_b13,model_b01,model_b03

In [ ]:
from scipy import stats

In [ ]:
cls_b01 = stats.binned_statistic_2d(df_in_time['tot_b0'].values, df_in_time['tot_b1'].values, 
                                    range=[[2,8],[4,8]], bins=(4,4),
                                    values=(nn_del_b01-(df_in_time['toa_b0']-df_in_time['toa_b1']).values), 
                                    statistic='mean')
res_b01 = stats.binned_statistic_2d(df_in_time['tot_b0'].values, df_in_time['tot_b1'].values, 
                                    range=[[2,8],[4,8]], bins=(4,4),
                                    values=(nn_del_b01-(df_in_time['toa_b0']-df_in_time['toa_b1']).values), 
                                    statistic='std')
plt.figure(dpi=50)
plt.imshow(cls_b01.statistic.T, extent=[cls_b01.x_edge[0], cls_b01.x_edge[-1], cls_b01.y_edge[0], cls_b01.y_edge[-1]],
           interpolation='nearest',aspect='auto',origin='lower')
plt.colorbar()
plt.show()
plt.figure(dpi=50)
plt.imshow(res_b01.statistic.T, extent=[res_b01.x_edge[0], res_b01.x_edge[-1], res_b01.y_edge[0], res_b01.y_edge[-1]],
           interpolation='nearest',aspect='auto',origin='lower')
plt.colorbar()
plt.show()

In [ ]:
del nn_del_b01,nn_del_b03,nn_del_b13

### Make the traditional plots with the NN outputs

In [ ]:
model_b01 = return_dense_model(numpars=2)
model_b03 = return_dense_model(numpars=2)
model_b13 = return_dense_model(numpars=2)
model_b01.load_weights(f'models/sep27_weights_{0}_b01.hdf5')
nn_del_b01 = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
model_b03.load_weights(f'models/sep27_weights_{0}_b03.hdf5')
nn_del_b03 = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
model_b13.load_weights(f'models/sep27_weights_{0}_b13.hdf5')
nn_del_b13 = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
for en_idx in range(1, ensemble_count):
    model_b01.load_weights(f'models/sep27_weights_{en_idx}_b01.hdf5')
    nn_del_b01 += model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
    model_b03.load_weights(f'models/sep27_weights_{en_idx}_b03.hdf5')
    nn_del_b03 += model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
    model_b13.load_weights(f'models/sep27_weights_{en_idx}_b13.hdf5')
    nn_del_b13 += model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
nn_del_b01 = nn_del_b01/ensemble_count
nn_del_b03 = nn_del_b03/ensemble_count
nn_del_b13 = nn_del_b13/ensemble_count
del model_b13,model_b01,model_b03

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction NN', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, -0.5*(nn_del_b01+nn_del_b03), 'r.', label='Neural Network')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction NN', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, 0.5*(nn_del_b01-nn_del_b13), 'r.', label='Neural Network')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction NN', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, 0.5*(nn_del_b03+nn_del_b13), 'r.', label='Neural Network')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
diff_b0_nn = df_in_time['toa_b0'].values - 0.5*(nn_del_b01+nn_del_b03)
diff_b1_nn = df_in_time['toa_b1'].values + 0.5*(nn_del_b01-nn_del_b13)
diff_b3_nn = df_in_time['toa_b3'].values + 0.5*(nn_del_b03+nn_del_b13)

diff_b01_nn = diff_b0_nn - diff_b1_nn
diff_b03_nn = diff_b0_nn - diff_b3_nn
diff_b13_nn = diff_b1_nn - diff_b3_nn

fit_params = []

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b01_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[0].hist(diff_b01_nn, range=(-1,1), bins=50, density=True, label='')
axes[0].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[0].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b03_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[1].hist(diff_b03_nn, range=(-1,1), bins=50, density=True, label='')
axes[1].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[1].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b13_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[2].hist(diff_b13_nn, range=(-1,1), bins=50, density=True, label='')
axes[2].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[2].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend()

plt.tight_layout()

In [ ]:
time_resol_b3 = np.sqrt(0.5)*(np.sqrt(fit_params[2][2]**2 + fit_params[1][2]**2 - fit_params[0][2]**2))
time_resol_b3*1e3

In [ ]:
time_resol_b1 = np.sqrt(0.5)*(np.sqrt(fit_params[2][2]**2 + fit_params[0][2]**2 - fit_params[1][2]**2))
time_resol_b1*1e3

In [ ]:
time_resol_b0 = np.sqrt(0.5)*(np.sqrt(fit_params[0][2]**2 + fit_params[1][2]**2 - fit_params[2][2]**2))
time_resol_b0*1e3

## Optimized Time Walk Correction -  Triple Input Neural Network

In [ ]:
def return_big_dense_model(numpars=3, print_summary=False):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(8, activation='relu', name='dense1',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(input)
    dense2 = Dense(8, activation='relu', name='dense2',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    output = Dense(1, activation='linear', name='output',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense2)
    model  = Model(inputs=[input], outputs=output, name="big_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    if(print_summary): print(model.summary())
    return model
model = return_big_dense_model(print_summary=True)
del model

In [ ]:
ensemble_count = 10

In [ ]:
do_training = False
if(do_training):
    for en_idx in range(ensemble_count):
        model_b130 = return_big_dense_model(numpars=1)
        checkpointer = ModelCheckpoint(f'models/sep28_weights_{en_idx}_b130.hdf5', verbose=0, save_best_only=True,monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
        history_b130 = model_b130.fit(
            df_in_time[['tot_b0']].values, 
            ((df_in_time['toa_b1']+df_in_time['toa_b3']).values/2.) - df_in_time['toa_b0'].values, 
            validation_split=0.4, 
            epochs=150,
            callbacks=[checkpointer,term,escb],
            verbose=0)
        del model_b130
        
        #plot the loss and validation loss of the dataset
        fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
        hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
        ax.set_title(f'Model {en_idx}, TWC: Board 0', loc="right", size=25)
        plt.plot(history_b130.history['loss'], label='mse')
        plt.plot(history_b130.history['val_loss'], label='val_mse')
        plt.yscale("log")
        plt.legend()
        plt.savefig("models/"+f"sep28_weights_{en_idx}_b130_loss"+".png")
        plt.show()

model_b130 = return_big_dense_model(numpars=1)
for en_idx in range(ensemble_count):
    model_b130.load_weights(f'models/sep28_weights_{en_idx}_b130.hdf5')
    if(en_idx==0): Y_pred = model_b130.predict(df_in_time[['tot_b0']].values, verbose=0).flatten()
    else: Y_pred += model_b130.predict(df_in_time[['tot_b0']].values, verbose=0).flatten()
del model_b130
Y_pred = Y_pred/ensemble_count
  
data_b0 = df_in_time['toa_b0'].values + Y_pred

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 0 Time Walk Correction NN', loc="right", size=25)
ax.scatter(df_in_time['tot_b0'].values,  ((df_in_time['toa_b1']+df_in_time['toa_b3']).values/2.) - df_in_time['toa_b0'].values, label='data')
ax.plot(df_in_time['tot_b0'].values, Y_pred, 'r.', label='Neural Network')
ax.set_xlabel('TOT time [ns]')
ax.set_ylabel(r'$(TOA_{1} + TOA_{3})/2 - TOA_0$ [ns]' )
ax.legend()

In [ ]:
do_training = True
if(do_training):
    for en_idx in range(ensemble_count):
        model_b031 = return_big_dense_model(numpars=1)
        checkpointer = ModelCheckpoint(f'models/sep28_weights_{en_idx}_b031.hdf5', verbose=0, save_best_only=True,monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
        history_b031 = model_b031.fit(
            df_in_time[['tot_b1']].values, 
            ((df_in_time['toa_b0']+df_in_time['toa_b3']).values/2.) - df_in_time['toa_b1'].values, 
            validation_split=0.4, 
            epochs=150,
            callbacks=[checkpointer,term,escb],
            verbose=0)
        del model_b031
        
        #plot the loss and validation loss of the dataset
        fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
        hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
        ax.set_title(f'Model {en_idx}, TWC: Board 1', loc="right", size=25)
        plt.plot(history_b031.history['loss'], label='mse')
        plt.plot(history_b031.history['val_loss'], label='val_mse')
        plt.yscale("log")
        plt.legend()
        plt.savefig("models/"+f"sep28_weights_{en_idx}_b031_loss"+".png")
        plt.show()

model_b031 = return_big_dense_model(numpars=1)
for en_idx in range(ensemble_count):
    model_b031.load_weights(f'models/sep28_weights_{en_idx}_b031.hdf5')
    if(en_idx==0): Y_pred = model_b031.predict(df_in_time[['tot_b1']].values, verbose=0).flatten()
    else: Y_pred += model_b031.predict(df_in_time[['tot_b1']].values, verbose=0).flatten()
del model_b031
Y_pred = Y_pred/ensemble_count
  
data_b1 = df_in_time['toa_b1'].values + Y_pred

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 1 Time Walk Correction NN', loc="right", size=25)
ax.scatter(df_in_time['tot_b1'].values,  ((df_in_time['toa_b0']+df_in_time['toa_b3']).values/2.) - df_in_time['toa_b1'].values, label='data')
ax.plot(df_in_time['tot_b1'].values, Y_pred, 'r.', label='Neural Network')
ax.set_xlabel('TOT time [ns]')
ax.set_ylabel(r'$(TOA_{0} + TOA_{3})/2 - TOA_1$ [ns]' )
ax.legend()

In [ ]:
do_training = True
if(do_training):
    for en_idx in range(ensemble_count):
        model_b013 = return_big_dense_model(numpars=1)
        checkpointer = ModelCheckpoint(f'models/sep28_weights_{en_idx}_b013.hdf5', verbose=0, save_best_only=True,monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
        history_b013 = model_b013.fit(
            df_in_time[['tot_b3']].values, 
            ((df_in_time['toa_b0']+df_in_time['toa_b1']).values/2.) - df_in_time['toa_b3'].values, 
            validation_split=0.4, 
            epochs=150,
            callbacks=[checkpointer,term,escb],
            verbose=0)
        del model_b013
        
        #plot the loss and validation loss of the dataset
        fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
        hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
        ax.set_title(f'Model {en_idx}, TWC: Board 3', loc="right", size=25)
        plt.plot(history_b013.history['loss'], label='mse')
        plt.plot(history_b013.history['val_loss'], label='val_mse')
        plt.yscale("log")
        plt.legend()
        plt.savefig("models/"+f"sep28_weights_{en_idx}_b013_loss"+".png")
        plt.show()

model_b013 = return_big_dense_model(numpars=1)
for en_idx in range(ensemble_count):
    model_b013.load_weights(f'models/sep28_weights_{en_idx}_b013.hdf5')
    if(en_idx==0): Y_pred = model_b013.predict(df_in_time[['tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b013.predict(df_in_time[['tot_b3']].values, verbose=0).flatten()
del model_b013
Y_pred = Y_pred/ensemble_count
  
data_b3 = df_in_time['toa_b3'].values + Y_pred

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 1 Time Walk Correction NN', loc="right", size=25)
ax.scatter(df_in_time['tot_b3'].values,  ((df_in_time['toa_b0']+df_in_time['toa_b1']).values/2.) - df_in_time['toa_b3'].values, label='data')
ax.plot(df_in_time['tot_b3'].values, Y_pred, 'r.', label='Neural Network')
ax.set_xlabel('TOT time [ns]')
ax.set_ylabel(r'$(TOA_{0} + TOA_{1})/2 - TOA_3$ [ns]' )
ax.legend()

In [ ]:
diff_b01_nn = data_b0 - data_b1
diff_b03_nn = data_b0 - data_b3
diff_b13_nn = data_b1 - data_b3

fit_params_NN = []
fit_errors_NN = []

In [ ]:
time_hist = hist.Hist(hist.axis.Regular(50, -1, 1, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
time_hist.fill(diff_b01_nn)
fig = plt.figure(figsize=(15, 10))
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])
main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
plt.setp(main_ax.get_xticklabels(), visible=False)
main_ax_artists, sublot_ax_arists = time_hist.plot_pull(
    "normal",
    eb_ecolor="steelblue",
    eb_mfc="steelblue",
    eb_mec="steelblue",
    eb_fmt="o",
    eb_ms=6,
    eb_capsize=1,
    eb_capthick=2,
    eb_alpha=0.8,
    fp_c="hotpink",
    fp_ls="-",
    fp_lw=2,
    fp_alpha=0.8,
    bar_fc="royalblue",
    pp_num=3,
    pp_fc="royalblue",
    pp_alpha=0.618,
    pp_ec=None,
    ub_alpha=0.2,
    fit_fmt= r"{name} = {value:.4g} $\pm$ {error:.4g}",
    ax_dict= {"main_ax":main_ax,"pull_ax":subplot_ax},
)
hep.cms.text(loc=0, ax=main_ax, text="Preliminary", fontsize=25)
main_ax.set_title(f'Board 0 - Board 1', loc="right", size=25)
plt.show()

In [ ]:
fit_params_NN.append([741.1, -0.325, 0.1123])
fit_errors_NN.append([12.85, 0.00156, 0.001173])

In [ ]:
time_hist = hist.Hist(hist.axis.Regular(50, -1, 1, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
time_hist.fill(diff_b03_nn)
fig = plt.figure(figsize=(15, 10))
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])
main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
plt.setp(main_ax.get_xticklabels(), visible=False)
main_ax_artists, sublot_ax_arists = time_hist.plot_pull(
    "normal",
    eb_ecolor="steelblue",
    eb_mfc="steelblue",
    eb_mec="steelblue",
    eb_fmt="o",
    eb_ms=6,
    eb_capsize=1,
    eb_capthick=2,
    eb_alpha=0.8,
    fp_c="hotpink",
    fp_ls="-",
    fp_lw=2,
    fp_alpha=0.8,
    bar_fc="royalblue",
    pp_num=3,
    pp_fc="royalblue",
    pp_alpha=0.618,
    pp_ec=None,
    ub_alpha=0.2,
    fit_fmt= r"{name} = {value:.4g} $\pm$ {error:.4g}",
    ax_dict= {"main_ax":main_ax,"pull_ax":subplot_ax},
)
hep.cms.text(loc=0, ax=main_ax, text="Preliminary", fontsize=25)
main_ax.set_title(f'Board 0 - Board 3', loc="right", size=25)
plt.show()

In [ ]:
fit_params_NN.append([831.9, -0.3442, 0.09955])
fit_errors_NN.append([14.53, 0.001384, 0.001056])

In [ ]:
time_hist = hist.Hist(hist.axis.Regular(50, -1, 1, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
time_hist.fill(diff_b13_nn)
fig = plt.figure(figsize=(15, 10))
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])
main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
plt.setp(main_ax.get_xticklabels(), visible=False)
main_ax_artists, sublot_ax_arists = time_hist.plot_pull(
    "normal",
    eb_ecolor="steelblue",
    eb_mfc="steelblue",
    eb_mec="steelblue",
    eb_fmt="o",
    eb_ms=6,
    eb_capsize=1,
    eb_capthick=2,
    eb_alpha=0.8,
    fp_c="hotpink",
    fp_ls="-",
    fp_lw=2,
    fp_alpha=0.8,
    bar_fc="royalblue",
    pp_num=3,
    pp_fc="royalblue",
    pp_alpha=0.618,
    pp_ec=None,
    ub_alpha=0.2,
    fit_fmt= r"{name} = {value:.4g} $\pm$ {error:.4g}",
    ax_dict= {"main_ax":main_ax,"pull_ax":subplot_ax},
)
hep.cms.text(loc=0, ax=main_ax, text="Preliminary", fontsize=25)
main_ax.set_title(f'Board 1 - Board 3', loc="right", size=25)
plt.show()

In [ ]:
fit_params_NN.append([899.9, -0.01758, 0.09358])
fit_errors_NN.append([15.36, 0.001289, 0.0009443])

In [ ]:
res_b3,err_b3 = return_resolution_ps(fit_params_NN[1][2], fit_errors_NN[1][2], 
                                     fit_params_NN[2][2], fit_errors_NN[2][2], 
                                     fit_params_NN[0][2], fit_errors_NN[0][2])
print(res_b3,err_b3)

In [ ]:
res_b1,err_b1 = return_resolution_ps(fit_params_NN[0][2], fit_errors_NN[0][2], 
                                     fit_params_NN[2][2], fit_errors_NN[2][2], 
                                     fit_params_NN[1][2], fit_errors_NN[1][2])
print(res_b1,err_b1)

In [ ]:
res_b0,err_b0 = return_resolution_ps(fit_params_NN[0][2], fit_errors_NN[0][2], 
                                     fit_params_NN[1][2], fit_errors_NN[1][2], 
                                     fit_params_NN[2][2], fit_errors_NN[2][2])
print(res_b0,err_b0)

In [ ]:
del diff_b01_nn,diff_b03_nn,diff_b13_nn,data_b3,data_b1,data_b0